In [1]:
!pip install transformers torchaudio torch librosa sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import SpeechEncoderDecoderModel, AutoFeatureExtractor, AutoTokenizer, GenerationConfig
import torchaudio
import torch
import librosa
import json


In [3]:
model_path = 'nguyenvulebinh/wav2vec2-bartpho'

model = SpeechEncoderDecoderModel.from_pretrained(model_path).eval()
feature_extractor = AutoFeatureExtractor.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

SpeechEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


RuntimeError: Failed to import transformers.models.wav2vec2.modeling_wav2vec2 because of the following error (look up to see its traceback):
/home/daoan/Projects/AI_Challenge_HCMC_2024/.venv/lib/python3.10/site-packages/flash_attn_2_cuda.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN2at4_ops5zeros4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEE

In [5]:
def decode_tokens(token_ids, skip_special_tokens=True):
    outputs = [[]]
    for token in token_ids:
        if token < tokenizer.vocab_size:  # Bỏ qua các token timestamp
            outputs[-1].append(token)
    outputs = [
         tokenizer.decode(s, skip_special_tokens=skip_special_tokens) for s in outputs
    ]

    return "".join(outputs).replace("<","").replace(">","")

In [ ]:
def speech_to_text(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    input_values = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = decode_tokens(predicted_ids[0])
    return transcription